In [ ]:
# Install h5py, cv2, telepot and Tensorflow 2.15
!pip install h5py
!pip install telepot
!pip install opencv-python
!pip install tensorflow==2.14.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 29.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.15.0
    Uninstalling tensorflow-estimator-2.15.0:
      Successfully uninstalled tensorflow-estimator-2.15.0
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 1.2.1
    Uninstalling google-auth-oauthlib-1.2.1:
      Successfully uninstalled google-auth-oauthlib-1.2.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboar

In [ ]:
import numpy as np
import cv2
import telepot
import tensorflow as tf
from tensorflow.keras.models import load_model
from collections import deque
from datetime import datetime
import pytz
from google.colab.patches import cv2_imshow
import h5py
import keras

print("TensorFlow Version:", tf.__version__)
print("Keras Version:", keras.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Step 3: Function to get current IST time
def getTime():
    IST = pytz.timezone('Asia/Kolkata')
    timeNow = datetime.now(IST)
    return timeNow

In [ ]:
import telepot

bot = telepot.Bot("7911172290:AAEp6Ar4OW5mQ04g6mDjgJDcKjm0YX2U1RM")

updates = bot.getUpdates()
print(updates)

In [ ]:
# Function to edit model to remove group tag
def edit_model(model_path):
  # Hack to change model config from keras 2->3 compliant
  f = h5py.File(model_path, mode="r+")
  model_config_string = f.attrs['model_config']
  model_config_string = model_config_string.replace('"groups": 1, ', '')
  model_config_string = model_config_string.replace("'groups': 1, ", "")
  del f.attrs['model_config']
  f.attrs['model_config'] = model_config_string
  f.close()

# Violence Detection Function
def detect_violence_in_video(video_path):
    trueCount = 0
    imageSaved = 0
    filename = 'savedImage.jpg'
    sendAlert = 0
    location = "Delhi NCR"

    print("Loading model ...")
    model_path = '/content/drive/MyDrive/Colab Notebooks/Alert System/finalmodel.h5'
    # Edit the model to remove the groups tag
    try:
      edit_model(model_path)
    except Exception as e:
      print("Error editing the model, please check model path, error is: ", e)

    # Try loading the model after edits
    try:
      model = load_model(model_path)
    except Exception as e:
      print("Error loading the model, please check model path, error is: ", e)
      return

    Q = deque(maxlen=128)
    vs = cv2.VideoCapture(video_path)
    writer = None
    (W, H) = (None, None)

    while True:
        (grabbed, frame) = vs.read()
        if not grabbed:
            break

        if W is None or H is None:
            (H, W) = frame.shape[:2]

        output = frame.copy()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = frame.reshape(128, 128, 3) / 255

        preds = model.predict(np.expand_dims(frame, axis=0))[0]
        Q.append(preds)

        results = np.array(Q).mean(axis=0)
        i = (preds > 0.50)[0]
        label = i

        text_color = (0, 255, 0)
        if label:
            text_color = (0, 0, 255)
            trueCount += 1

        text = "Violence: {}".format(label)
        FONT = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3)

        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter("recordedVideo.avi", fourcc, 30, (W, H), True)

        writer.write(output)
        cv2_imshow(output)

        if trueCount == 50:
            if imageSaved == 0:
                if label:
                    cv2.imwrite(filename, output)
                    imageSaved = 1

            if sendAlert == 0:
                timeMoment = getTime()
                bot = telepot.Bot('7911172290:AAEp6Ar4OW5mQ04g6mDjgJDcKjm0YX2U1RM')
                bot.sendMessage(-1002448977692, f"VIOLENCE ALERT!! \nLOCATION: {location} \nTIME: {timeMoment}")
                bot.sendPhoto(-1002448977692, photo=open(filename, 'rb'))
                sendAlert = 1

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    print("[INFO] cleaning up...")
    writer.release()
    vs.release()

# Define video paths
V_path = "/content/drive/MyDrive/Colab Notebooks/Alert System/Real Life Violence Dataset/Violence/V_15.mp4"
NV_path = "/content/drive/MyDrive/Colab Notebooks/Alert System/Real Life Violence Dataset/NonViolence/NV_10.mp4"

# Run the violence detection on the video
detect_violence_in_video(V_path)